In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import seaborn as sns
from IPython.display import YouTubeVideo
import matplotlib.pyplot as plt
import plotly.plotly as py

from pyspark import SparkConf, SparkContext
from pyspark.sql.types import *
import pyspark
import numpy as np
from elephas.spark_model import SparkModel
from create_and_train_biLSTM_Youtube import create_model

import os
from glob import glob
from tqdm import tqdm
import sys

WARNING


Using TensorFlow backend.


In [2]:
conf = SparkConf().setAppName('Youtube-8M') \
                  .set("spark.jars",
                       "ecosystem/spark/spark-tensorflow-connector/target/spark-tensorflow-connector_2.11-1.10.0.jar")
sc = SparkContext(conf = conf)
spark = pyspark.sql.SparkSession(sc)

In [3]:
top_dir = "yt8pm_100th_shard/v2"
data_path = lambda level, set_name: "{}/{}/{}*.tfrecord".format(top_dir, level, set_name)

# just work with ONE TF record for now.

# VIDEO-LEVEL
#vid_train_df = spark.read.format("tfrecords").option("recordType", "Example").load(data_path('video','train'))

vid_train_df = spark.read.format("tfrecords").option("recordType", "Example").load("mys3bucket/yt8pm_100th_shard/v2/video/train0093.tfrecord")




In [9]:
from pyspark.sql.types import *

    
fields = [StructField("id", StringType()),
          StructField("labels", ArrayType(IntegerType(), True)),
          StructField("rgb", ArrayType(ArrayType(StringType(), True),True)),
          StructField("audio", ArrayType(ArrayType(StringType(), True),True))]
                      

schema = StructType(fields)

In [14]:
schema

StructType(List(StructField(id,StringType,true),StructField(labels,ArrayType(IntegerType,true),true),StructField(rgb,ArrayType(ArrayType(StringType,true),true),true),StructField(audio,ArrayType(ArrayType(StringType,true),true),true)))

In [15]:

# FRAME-LEVEL
#frame_train_df = spark.read.format("tfrecords").option("recordType", "SequenceExample").load(data_path('frame','train'))
sess = tf.InteractiveSession()
frame_train_df = spark.read.format("tfrecords").schema(schema).option("recordType", "SequenceExample").load("mys3bucket/yt8pm_100th_shard/v2/frame/train0093.tfrecord")
tf.reset_default_graph()
sess.close()

In [17]:
audio = frame_train_df.select('audio').collect()

In [24]:
import struct

In [33]:
[ a.decode("utf-8") for a in audio[0][0][0]]

AttributeError: 'str' object has no attribute 'decode'

In [6]:
vid_train_df.show()

+--------------------+--------------------+--------------------+----+
|          mean_audio|              labels|            mean_rgb|  id|
+--------------------+--------------------+--------------------+----+
|[-1.2556146, 0.17...|             [0, 12]|[0.5198898, 0.301...|eXbF|
|[-0.32460678, -0....|[16, 25, 189, 645...|[-0.5026991, -1.6...|BFbF|
|[-1.7352352, 1.83...|[2, 44, 64, 113, ...|[0.24258906, 0.97...|GqbF|
|[0.7349236, 1.268...|                 [3]|[-0.026906455, -0...|XabF|
|[1.2375641, -0.14...|              [1, 5]|[-0.45482802, -1....|3mbF|
|[0.50689745, 0.02...|                [14]|[0.45552492, 0.64...|S6bF|
|[0.71223485, 1.41...|      [3, 4, 13, 54]|[-0.022711225, 0....|mXbF|
|[-0.662256, -0.97...|           [11, 579]|[0.3059462, 0.947...|7sbF|
|[0.9852263, 0.145...|[2, 76, 227, 474,...|[0.24360302, 0.40...|H1bF|
|[0.10193015, -0.9...| [49, 80, 265, 2063]|[0.60755104, 0.43...|fxbF|
|[0.2003511, -1.10...|     [0, 1, 36, 132]|[-0.4298068, -1.0...|w1bF|
|[-0.5392725, -0.9..

+----+--------------------+--------------------+--------------------+
|  id|              labels|                 rgb|               audio|
+----+--------------------+--------------------+--------------------+
|lKbF| [1, 19, 2852, 3477]|[WrappedArray(i�^...|[WrappedArray(�C�...|
|HebF|              [3, 6]|[WrappedArray(��...|[WrappedArray(���...|
|GkbF|                [66]|[WrappedArray(��m...|[WrappedArray(�K�...|
|q2bF|                [14]|[WrappedArray( H�...|[WrappedArray(>� ...|
|aUbF|     [141, 247, 255]|[WrappedArray(!bb...|[WrappedArray(�~...|
|A9bF|[2, 17, 30, 55, 174]|[WrappedArray(��B...|[WrappedArray(�~...|
|52bF|                [25]|[WrappedArray( ��...|[WrappedArray(�]�...|
|gNbF|              [2, 7]|[WrappedArray( H�...|[WrappedArray(�+s...|
|6ZbF|       [1, 246, 658]|[WrappedArray(5.[...|[WrappedArray(e��...|
|rubF|            [39, 66]|[WrappedArray(a�u...|[WrappedArray(g<�...|
|UqbF|                [11]|[WrappedArray( z/...|[WrappedArray(�~...|
|ATbF|   [0, 1, 245,

In [ ]:
# how to convert wrapped array???

In [6]:
# quickly check we have agreement on the number of videos in video-level and feature-level IDs
frame_ids = frame_train_df.select('id')
video_ids = vid_train_df.select('id')
video_ids = video_ids.toPandas()
frame_ids = frame_ids.toPandas()

In [7]:
video_ids.shape,frame_ids.shape

((1015, 1), (1015, 1))

In [8]:
video_ids = video_ids.id.tolist()
frame_ids = frame_ids.id.tolist()
assert len(set(video_ids).intersection(set(frame_ids))) == frame_train_df.count()

In [9]:
# ok, as seen above, the TF records contain same videos on video and feature level for each TF Record

In [10]:
# drop labels from one dataframe - its duplicate!
frame_train_df = frame_train_df.drop('labels')

In [11]:
# FILIP NOTE: will this traditional SQL-like join work in memory on the whole thing?
# we have two options here: either perform all the operations with SQL or in RDD
# we will stick to SQL for now since IT IS MUCH EASIER.

total_spark_df = vid_train_df.join(frame_train_df,on='id',how='inner')

In [12]:
total_spark_df.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  id|          mean_audio|              labels|            mean_rgb|               audio|                 rgb|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+
|3cbF|[1.0291835, -0.32...|       [10, 97, 243]|[-0.1551302, 0.97...|[WrappedArray(�~...|[WrappedArray( H�...|
|KtbF|[1.175444, 1.8621...|       [3, 4, 6, 38]|[-0.8292868, 0.38...|[WrappedArray(���...|[WrappedArray(��...|
|TjbF|[1.0009577, -0.45...|[2, 17, 76, 1068,...|[-0.59980744, 0.0...|[WrappedArray(�m�...|[WrappedArray(\�<...|
|UXbF|[-1.1192986, -0.5...|[15, 60, 239, 363...|[0.38621774, 0.99...|[WrappedArray(�~...|[WrappedArray( H�...|
|bobF|[0.22682534, 1.62...| [82, 103, 118, 329]|[0.5607807, 0.684...|[WrappedArray(��֬...|[WrappedArray(�q�...|
|nKbF|[-0.025651552, -0...|[11, 20, 22, 29, ...|[0.012622957, 1.0...|[WrappedArray(qXݗ...|[WrappedArray(

In [13]:
total_spark_df.count()

1015

In [14]:
train_df = total_spark_df.select('mean_audio','mean_rgb','audio','rgb','labels')

In [15]:
train_df.columns

['mean_audio', 'mean_rgb', 'audio', 'rgb', 'labels']

In [56]:
df = total_spark_df.select('audio').toPandas()

In [57]:
df.shape

(1015, 1)

In [61]:
df.iloc[0][0][0]

['�\x1b~_�YC����Dq�p\x19d���\x00�^�P�q��.�vpF����϶m�D�~��f�GzO�=P�M�g�h��g�yc�t�|imhx~��Z~��l�qyqvlu��p�y���m��t��~�ilp��z��syf}ruy~~�r���']

In [16]:
# def one_hot_y(raw_labels,label_size=20):
#     '''
#     Helper function to transform labels into one-hot TOP 20
#     Uses np.unique(return_counts=True) as implicit sorter (first K labels are the most frequent)
#     '''
#     all_labels = []
#     for i in list(raw_labels):
#         for j in list(i):
#             all_labels.append(j)

#     results = np.unique(all_labels,return_counts=True)
#     labels_vocab,counts = results

#     labels = labels_vocab[:label_size-1] #last columns will be 1 if none of those labels found in a video
#     output = []
#     for set_of_labels in raw_labels:
        
#         # preallocate numpy arr for each set of labels
#         sequence = np.zeros(label_size)
#         # loop through all the labels in one video and flip them to 1s
#         for this_label in set_of_labels:
#             designation = np.where(labels==this_label)
#             for des in designation:
#                 sequence[des]=1
#         # done with one training points
#         if sequence.sum()==0:
#             sequence[-1]=1
#         output.append(sequence)

#     return output

In [25]:
# from pyspark.sql.functions import udf

In [26]:
# from pyspark.sql.types import ArrayType

In [41]:
# udf_one_hot_y = udf(one_hot_y, ArrayType(LongType()))

In [42]:
# train_df = train_df.withColumn('labels',udf_one_hot_y(train_df['labels']))

In [38]:
# labels_df = train_df.select('labels')

In [39]:
# labels_rdd = labels_df.rdd

In [54]:
# labels_rdd.take(1)

In [21]:
train_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|          mean_audio|            mean_rgb|               audio|                 rgb|              labels|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[1.0291835, -0.32...|[-0.1551302, 0.97...|[WrappedArray(�~...|[WrappedArray( H�...|       [10, 97, 243]|
|[1.175444, 1.8621...|[-0.8292868, 0.38...|[WrappedArray(���...|[WrappedArray(��...|       [3, 4, 6, 38]|
|[1.0009577, -0.45...|[-0.59980744, 0.0...|[WrappedArray(�m�...|[WrappedArray(\�<...|[2, 17, 76, 1068,...|
|[-1.1192986, -0.5...|[0.38621774, 0.99...|[WrappedArray(�~...|[WrappedArray( H�...|[15, 60, 239, 363...|
|[0.22682534, 1.62...|[0.5607807, 0.684...|[WrappedArray(��֬...|[WrappedArray(�q�...| [82, 103, 118, 329]|
|[-0.025651552, -0...|[0.012622957, 1.0...|[WrappedArray(qXݗ...|[WrappedArray( R�...|[11, 20, 22, 29, ...|
|[1.1240029, 0.165...|[0.9078685, -0.

In [22]:
def convert_labels(labels):
    allowed=np.arange(20)
    one_hot=np.zeros(20)
    for l in labels:
        if l in allowed:
            one_hot[l]=1
    return one_hot

# to-do: create a list of top 1000 labels

In [40]:
def convert_frame(video_frames,max_frame_sequence_length=10):    
    return np.vstack(frame)[:max_frame_sequence_length,:]

In [23]:
train_rdd = train_df.rdd

In [43]:
# order needed: video rgb, video, audio
train_rdd_sample = train_rdd.map(lambda x: (np.array(x[1]),np.array(x[0]),x[2],convert_labels(x[4])))

In [44]:
one_rwo = train_rdd_sample.take(1)

In [53]:
type(one_rwo[0][2][0][0])

str

## Bi-LSTM video classification

In [13]:
# keras imports
from keras.layers import Dense, Input, LSTM, Dropout, Bidirectional
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.layers.embeddings import Embedding
from keras.layers.merge import concatenate
from keras.callbacks import TensorBoard
from keras.models import load_model
from keras.models import Model
import operator
import time 
import gc
import os

Using TensorFlow backend.


In [14]:
## creating training and dev set

In [15]:
## Defining Model parameters and creating architecture

In [16]:

# 10 class problem for now?
label_feature_size = 10

# how many frames we will use from each video?
max_frame_rgb_sequence_length = 10
frame_rgb_embedding_size = 1024

# how many audio sequences we will use from each video?
max_frame_audio_sequence_length = 10
frame_audio_embedding_size = 128

number_dense_units = 1000
number_lstm_units = 100
rate_drop_lstm = 0.2
rate_drop_dense = 0.2
activation_function='relu'
validation_split_ratio = 0.2


def create_model():
    """Create and store best model at `checkpoint` path ustilising bi-lstm layer for frame level data of videos"""
    
    # Creating 2 bi-lstm layer, one for rgb and other for audio level data
    lstm_layer_1 = Bidirectional(LSTM(number_lstm_units, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
    lstm_layer_2 = Bidirectional(LSTM(number_lstm_units, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm))
    
    # creating input layer for frame-level data
    frame_rgb_sequence_input = Input(shape=(max_frame_rgb_sequence_length, frame_rgb_embedding_size), dtype='float32')
    frame_audio_sequence_input = Input(shape=(max_frame_audio_sequence_length, frame_audio_embedding_size), dtype='float32')
    frame_x1 = lstm_layer_1(frame_rgb_sequence_input)
    frame_x2 = lstm_layer_2(frame_audio_sequence_input)
    
    #creating input layer for video-level data 
    vid_shape=(1024,)
    video_rgb_input = Input(shape=vid_shape)
    video_rgb_dense = Dense(int(number_dense_units/2), activation=activation_function, input_shape=vid_shape)(video_rgb_input)
    
    aud_shape=(128,)
    video_audio_input = Input(shape=aud_shape)
    video_audio_dense = Dense(int(number_dense_units/2), activation=activation_function,input_shape = aud_shape)(video_audio_input)
    
    # merging frame-level bi-lstm output and later passed to dense layer by applying batch-normalisation and dropout
    merged_frame = concatenate([frame_x1, frame_x2])
    merged_frame = BatchNormalization()(merged_frame)
    merged_frame = Dropout(rate_drop_dense)(merged_frame)
    merged_frame_dense = Dense(int(number_dense_units/2), activation=activation_function)(merged_frame)
    
    # merging video-level dense layer output
    merged_video = concatenate([video_rgb_dense, video_audio_dense])
    merged_video = BatchNormalization()(video_rgb_dense)
    merged_video = Dropout(rate_drop_dense)(merged_video)
    merged_video_dense = Dense(int(number_dense_units/2), activation=activation_function)(merged_video)

    
    # merging frame-level and video-level dense layer output
    merged = concatenate([merged_frame_dense, merged_video_dense])
    merged = BatchNormalization()(merged)
    merged = Dropout(rate_drop_dense)(merged)
     
    merged = Dense(number_dense_units, activation=activation_function)(merged)
    merged = BatchNormalization()(merged)
    merged = Dropout(rate_drop_dense)(merged)
    preds = Dense(label_feature_size, activation='sigmoid')(merged)
    
    model = Model(inputs=[frame_rgb_sequence_input, frame_audio_sequence_input, video_rgb_input, video_audio_input], outputs=preds)

    print(model.summary())
    
    model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['acc'])

    return model

In [17]:
model = create_model()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 1024)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 10, 128)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 200)          900000      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidi

In [18]:
len(train_vid_ids), len(train_labels), len(train_mean_rgb), len(train_mean_audio)

(41393, 41393, 41393, 41393)

In [19]:
# we only grabbed 5 videos, it took 6 minutes to load!!
len(train_frame_ids),len(train_frame_labels),len(train_frame_rgb),len(train_frame_audio)

(20, 20, 20, 20)

In [20]:
list(set(train_frame_ids).intersection(set(train_vid_ids)))

['ATbF',
 'q2bF',
 'exbF',
 'gNbF',
 'GkbF',
 'aUbF',
 'A9bF',
 'UGbF',
 'pUbF',
 'DLbF',
 '52bF',
 'CsbF',
 '3lbF',
 '6ZbF',
 'rubF',
 'UqbF',
 'HebF',
 'oJbF',
 'lKbF',
 'CFbF']

In [21]:
list(set(train_frame_ids).intersection(set(train_vid_ids)))

['ATbF',
 'q2bF',
 'exbF',
 'gNbF',
 'GkbF',
 'aUbF',
 'A9bF',
 'UGbF',
 'pUbF',
 'DLbF',
 '52bF',
 'CsbF',
 '3lbF',
 '6ZbF',
 'rubF',
 'UqbF',
 'HebF',
 'oJbF',
 'lKbF',
 'CFbF']

In [22]:
def create_train_dev_dataset(video_rgb, video_audio, vid_ids, frame_rgb, frame_audio, frame_labels, frame_ids):
    """
    Method to created training and validation data. 
    We need to make sure we only use video IDs for which we have frames.
    This is handled below.
    
    """
    # we have to have the same video of for both video and frame-level features
    video_rgb_matching = []
    video_audio_matching = []
    
    for idx in frame_ids: # for each ID available on frame level, find matching video-level features
        for i, idx_vid in enumerate(vid_ids): # scan through video-level ids
            if idx == idx_vid: 
                video_rgb_matching.append(video_rgb[i])
                video_audio_matching.append(video_audio[i])
                
                
    shuffle_indices = np.random.permutation(np.arange(len(frame_labels)))
        
    video_rgb_shuffled = np.array(video_rgb_matching)[shuffle_indices]
    video_audio_shuffled = np.array(video_audio_matching)[shuffle_indices]
    frame_rgb_shuffled = np.array(frame_rgb)[shuffle_indices]
    frame_audio_shuffled = np.array(frame_audio)[shuffle_indices]
    labels_shuffled = np.array(frame_labels)[shuffle_indices]

    dev_idx = max(1, int(len(labels_shuffled) * validation_split_ratio))
    
    # delete orig vars to clear some cache
    del video_rgb
    del video_audio
    del frame_rgb
    del frame_audio
    gc.collect()
    
    train_video_rgb, val_video_rgb = video_rgb_shuffled[:-dev_idx], video_rgb_shuffled[-dev_idx:]
    train_video_audio, val_video_audio = video_audio_shuffled[:-dev_idx], video_audio_shuffled[-dev_idx:]
    
    train_frame_rgb, val_frame_rgb = frame_rgb_shuffled[:-dev_idx], frame_rgb_shuffled[-dev_idx:]
    train_frame_audio, val_frame_audio = frame_audio_shuffled[:-dev_idx], frame_audio_shuffled[-dev_idx:]
    
    train_labels, val_labels = labels_shuffled[:-dev_idx], labels_shuffled[-dev_idx:]
    
    del video_rgb_shuffled, video_audio_shuffled, frame_rgb_shuffled, frame_audio_shuffled, labels_shuffled
    gc.collect()
    
    return (train_video_rgb, train_video_audio, train_frame_rgb, train_frame_audio, train_labels, val_video_rgb, val_video_audio, 
            val_frame_rgb, val_frame_audio, val_labels)

In [23]:
train_video_rgb, train_video_audio, train_frame_rgb, train_frame_audio, \
train_labels, val_video_rgb, val_video_audio, val_frame_rgb, val_frame_audio, val_labels \
                = create_train_dev_dataset(train_mean_rgb, train_mean_audio, train_vid_ids, train_frame_rgb, \
                                            train_frame_audio, train_frame_labels, train_frame_ids )

In [24]:
# transform into final input in the model
def one_hot_y(raw_labels=train_labels,label_size=20):
    '''
    Helper function to transform labels into one-hot TOP 20
    Uses np.unique(return_counts=True) as implicit sorter (first K labels are the most frequent)
    '''
    all_labels = []
    for i in list(raw_labels):
        for j in list(i):
            all_labels.append(j)

    results = np.unique(all_labels,return_counts=True)
    labels_vocab,counts = results

    labels = labels_vocab[:label_size-1] #last columns will be 1 if none of those labels found in a video
    output = []
    for set_of_labels in raw_labels:
        
        # preallocate numpy arr for each set of labels
        sequence = np.zeros(label_size)
        # loop through all the labels in one video and flip them to 1s
        for this_label in set_of_labels:
            designation = np.where(labels==this_label)
            for des in designation:
                sequence[des]=1
        # done with one training points
        if sequence.sum()==0:
            sequence[-1]=1
        output.append(sequence)
    return output


In [25]:
def transform_data_for_lstm(video_rgb,video_audio, frame_rgb, frame_audio,
                            labels,label_feature_size=10,max_frame_rgb_sequence_length = 10,\
                            max_frame_audio_sequence_length = 10):
    frames = []
    # need to transfrom to numpy (num_videos x max_frame_rgb_sequence_length x 1024)
    #print(len(frame_rgb))
    
    for frame in frame_rgb: 
        # stack the frames in each video, only allowed number of first frams
        #print(np.vstack(frame).shape)
        frames.append(np.vstack(frame)[:max_frame_rgb_sequence_length,:])
    #print(len(frames))

    frames = np.reshape(np.array(frames),(len(frame_rgb),max_frame_rgb_sequence_length,1024))

    #print(frames.shape)
    
    frames_audio = []
    # need to transfrom to numpy (num_videos x max_frame_audio_sequence_length x 128)
    for frame in frame_audio:
        # stack the frames in each video, only allowed number of first frams
        #print(np.vstack(frame).shape)
        frames_audio.append(np.vstack(frame)[:max_frame_audio_sequence_length,:])

    frames_audio = np.reshape(np.array(frames_audio),(len(frame_audio),max_frame_audio_sequence_length,128))
    #print(frames_audio.shape)
    
    # deal with videos
    
    video_rgb = np.vstack(video_rgb)
    video_audio = np.vstack(video_audio)
    
    
    # labels - need to one-hot encode TOP - K label
    labels = one_hot_y(labels,label_feature_size)
    labels = np.vstack(labels)
    return frames,frames_audio, video_rgb,video_audio, labels

In [28]:
train_frame_rgb, train_frame_audio, train_video_rgb, train_video_audio, train_labels = \
    transform_data_for_lstm(train_video_rgb, train_video_audio, train_frame_rgb, train_frame_audio,train_labels)

In [29]:
train_frame_rgb.shape, train_frame_audio.shape, train_video_rgb.shape, train_video_audio.shape,train_labels.shape

((16, 10, 1024), (16, 10, 128), (16, 1024), (16, 128), (16, 10))

In [30]:
train_labels[0]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [31]:
val_frame_rgb, val_frame_audio, val_video_rgb, val_video_audio, val_labels = \
    transform_data_for_lstm( val_video_rgb, val_video_audio,val_frame_rgb, val_frame_audio, val_labels)

In [32]:
val_frame_rgb.shape, val_frame_audio.shape, val_video_rgb.shape, val_video_audio.shape,val_labels.shape

((4, 10, 1024), (4, 10, 128), (4, 1024), (4, 128), (4, 10))

In [33]:
val_labels[0]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

In [34]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
    
STAMP = 'lstm_%d_%d_%.2f_%.2f' % (number_lstm_units, number_dense_units, rate_drop_lstm, rate_drop_dense)

checkpoint_dir = 'checkpoints/' + str(int(time.time())) + '/'

if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

bst_model_path = checkpoint_dir + STAMP + '.h5'
model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)
tensorboard = TensorBoard(log_dir=checkpoint_dir + "logs/{}".format(time.time()))

model.fit([train_frame_rgb, train_frame_audio, train_video_rgb, train_video_audio], train_labels,
          validation_data=([val_frame_rgb, val_frame_audio, val_video_rgb, val_video_audio], val_labels),
          epochs=200, batch_size=1, shuffle=True, callbacks=[early_stopping, model_checkpoint, tensorboard]) 

Instructions for updating:
Use tf.cast instead.
Train on 16 samples, validate on 4 samples
Epoch 1/200
16/16 [==============================] - 7s 408ms/step - loss: 3.8839 - acc: 0.3750 - val_loss: 18.3893 - val_acc: 0.2500
Epoch 2/200
16/16 [==============================] - 1s 91ms/step - loss: 3.8130 - acc: 0.4375 - val_loss: 18.3122 - val_acc: 0.2500
Epoch 3/200
16/16 [==============================] - 1s 91ms/step - loss: 3.7296 - acc: 0.4375 - val_loss: 18.2211 - val_acc: 0.2500
Epoch 4/200
16/16 [==============================] - 1s 92ms/step - loss: 3.6801 - acc: 0.4375 - val_loss: 18.2211 - val_acc: 0.2500
Epoch 5/200
16/16 [==============================] - 1s 92ms/step - loss: 3.7411 - acc: 0.4375 - val_loss: 18.2666 - val_acc: 0.2500
Epoch 6/200
16/16 [==============================] - 1s 90ms/step - loss: 3.6986 - acc: 0.4375 - val_loss: 18.2108 - val_acc: 0.2500
Epoch 7/200
16/16 [==============================] - 1s 92ms/step - loss: 3.6579 - acc: 0.4375 - val_loss: 18.